In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import numpy as np
import talib as ta

In [7]:
activo = 'BTC-USD'
df = yf.Ticker(activo)
df = df.history(start='2018-01-01', end='2024-11-28')

In [8]:
def backtest_macd_strategy(fastperiod, slowperiod, signalperiod):
    df['MACD'], df['MACD_Signal'], _ = ta.MACD(df['Close'],
                                               fastperiod=fastperiod,
                                               slowperiod=slowperiod,
                                               signalperiod=signalperiod)
    df['Signal'] = np.where(df['MACD'] > df['MACD_Signal'], 1, -1)
    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    
    return (df['Strategy_Returns'] + 1).cumprod().iloc[-1] - 1 # Retorno total

In [9]:
# Definir el rango de parámetros a probar
fast_periods = range(2, 50, 1)    # Período rápido
slow_periods = range(10, 200, 1)  # Período lento
signal_periods = range(2, 50, 1)  # Período de señal

In [10]:
# Probar todas las combinaciones de parámetros
best_performance = -np.inf
best_params = (None, None, None)

In [11]:
for fast_period in fast_periods:
    for slow_period in slow_periods:
        if fast_period >= slow_period:
            continue  # Evitar combinaciones donde el período rápido sea mayor o igual al lento
        for signal_period in signal_periods:
            performance = backtest_macd_strategy(fast_period, slow_period, signal_period)
            if performance > best_performance:
                best_performance = performance
                best_params = (fast_period, slow_period, signal_period)


print(f"Mejores parámetros: MACD rápido={best_params[0]}, MACD lento={best_params[1]}, Señal={best_params[2]}")
print(f"Mejor rendimiento: {best_performance:.2%}")

Mejores parámetros: MACD rápido=3, MACD lento=118, Señal=27
Mejor rendimiento: 7174.67%
